In [ ]:
import requests
import pandas as pd
from data_processor import TennisDataProcessor, to_player, to_average,get_fatigue_stats

ROUND_ORDER = ["RR", "R128", "R64", "R32", "R16", "QF", "SF", "F"]

pd.options.display.max_columns = None
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_matches_{}.csv"
data_20s = pd.concat([pd.read_csv(url.format(year)) for year in range(2000, 2025)])


In [ ]:
tdp = TennisDataProcessor(data_20s.copy())
derived_data = tdp.derive_match_data()

In [ ]:
roger_data = to_player("Roger Federer", derived_data)
roger_averages = to_average(roger_data, lookback=10)
roger_full = get_fatigue_stats(roger_averages)